### Target Challenge
#### Silicon Slayers


### 1. Target
Our target is the number of people attending church for each census tract. We start with filtering the data frame down to just Churches by filtering by location name. We also take out the temples by using their specific addresses. Next we take our newly filtered data and organize it by census tract. We can do this because the data gives us the home census tract for each visitor at the church buildings. After that we calculate Sunday attendance by multiplying the total visitors for each census tract by the ratio of Sunday visits compared to the rest of the week. Finally we verify our results by making sure we have 298 tracts(the number of census tracts in Idaho) and graphing our results.

### 2. Pseudocode
- Load data and packages
- Filter places for LDS churches with reg ex 
- Filter out the Temples
- Join Filter places table to patterns table
- Isolate the month from the date_range_start column and create a month column
- Select needed columns and explode the visitor_home_aggregation
- Create scaled visitors column(logic is tract_visitors * (normalized_visits_by_state_scaling/raw_visit_counts))
- Collect the ratio of Sunday visits to the other weekdays and multiply that number to our new scaled visitors
- Group by tract and month and sum total scaled visitors
- Plot and explore!

In [0]:
# Load libraries
import pyspark.sql.functions as F
import requests
import shutil
import pandas as pd
import plotly.express as px

In [0]:
# Read data
patterns = spark.read.parquet('dbfs:/data/idaho/patterns')
places = spark.read.parquet('dbfs:/data/idaho/places.parquet')
temples = spark.read.parquet('dbfs:/FileStore/temple_details_2.parquet')

Filter to only the Church of Jesus Christ of Latter day Saints

In [0]:
# 384 Rows (exact match), 406 Rows (levenshtein, LDS matches, and excluding temples)
# My original code, replaced by Hathaway's
my_churches = places.filter(
    (places.location_name == 'LDS Church') | 
    (places.location_name == 'Church of Jesus Christ LDS') |
    (F.levenshtein(F.lit("The Church of Jesus Christ of Latter day Saints"), F.col("location_name")) < 10)
)

In [0]:
idaho_temples = places.filter(
    ((places.street_address == '750 S 2nd E') & (places.city == 'Rexburg')) | 
    ((places.street_address == '1000 Memorial Dr') & (places.city == 'Idaho Falls')) | 
    ((places.street_address == '3100 Butte St') & (places.city == 'Pocatello')) | 
    ((places.street_address == '1405 Eastland Dr N') & (places.city == 'Twin Falls')) | 
    ((places.street_address == '1211 S Cole Rd') & (places.city == 'Boise')) |
    ((places.street_address == '7355 N Linder Rd') & (places.city == 'Meridian'))
)

In [0]:
hathaway_churches = places.filter(
    (F.col("top_category") == "Religious Organizations") &
    (F.col("location_name").rlike("Latter|latter|Saints|saints|LDS|\b[Ww]ard\b")) &
    (F.col("location_name").rlike("^((?!Reorganized).)*$")) &
    (F.col("location_name").rlike("^((?!All Saints).)*$")) &
    (F.col("location_name").rlike("^((?![cC]ath).)*$")) &
    (F.col("location_name").rlike("^((?![Bb]ody).)*$")) &
    (F.col("location_name").rlike("^((?![Pp]eter).)*$")) &
    (F.col("location_name").rlike("^((?![Cc]atholic).)*$")) &
    (F.col("location_name").rlike("^((?![Pp]res).)*$")) &
    (F.col("location_name").rlike("^((?![Mm]inist).)*$")) &
    (F.col("location_name").rlike("^((?![Mm]ission).)*$")) &
    (F.col("location_name").rlike("^((?![Ww]orship).)*$")) &
    (F.col("location_name").rlike("^((?![Rr]ain).)*$")) &
    (F.col("location_name").rlike("^((?![Bb]aptist).)*$")) &
    (F.col("location_name").rlike("^((?![Mm]eth).)*$")) &
    (F.col("location_name").rlike("^((?![Ee]vang).)*$")) &
    (F.col("location_name").rlike("^((?![Ll]utheran).)*$")) &
    (F.col("location_name").rlike("^((?![Oo]rthodox).)*$")) &
    (F.col("location_name").rlike("^((?![Ee]piscopal).)*$")) &
    (F.col("location_name").rlike("^((?![Tt]abernacle).)*$")) &
    (F.col("location_name").rlike("^((?![Hh]arvest).)*$")) &
    (F.col("location_name").rlike("^((?![Aa]ssem).)*$")) &
    (F.col("location_name").rlike("^((?![Mm]edia).)*$")) &
    (F.col("location_name").rlike("^((?![Mm]artha).)*$")) &
    (F.col("location_name").rlike("^((?![Cc]hristian).)*$")) &
    (F.col("location_name").rlike("^((?![Uu]nited).)*$")) &
    (F.col("location_name").rlike("^((?![Ff]ellowship).)*$")) &
    (F.col("location_name").rlike("^((?![Ww]esl).)*$")) &
    (F.col("location_name").rlike("^((?![C]cosmas).)*$")) &
    (F.col("location_name").rlike("^((?![Gg]reater).)*$")) &
    (F.col("location_name").rlike("^((?![Pp]rison).)*$")) &
    (F.col("location_name").rlike("^((?![Cc]ommuni).)*$")) &
    (F.col("location_name").rlike("^((?![Cc]lement).)*$")) &
    (F.col("location_name").rlike("^((?![Vv]iridian).)*$")) &
    (F.col("location_name").rlike("^((?![Dd]iocese).)*$")) &
    (F.col("location_name").rlike("^((?![Hh]istory).)*$")) &
    (F.col("location_name").rlike("^((?![Ss]chool).)*$")) &
    (F.col("location_name").rlike("^((?![Tt]hougt).)*$")) &
    (F.col("location_name").rlike("^((?![Hh]oliness).)*$")) &
    (F.col("location_name").rlike("^((?![Mm]artyr).)*$")) &
    (F.col("location_name").rlike("^((?![Jj]ames).)*$")) &
    (F.col("location_name").rlike("^((?![Ff]ellowship).)*$")) &
    (F.col("location_name").rlike("^((?![Hh]ouse).)*$")) &
    (F.col("location_name").rlike("^((?![Gg]lory).)*$")) &
    (F.col("location_name").rlike("^((?![Aa]nglican).)*$")) &
    (F.col("location_name").rlike("^((?![Pp]oetic).)*$")) &
    (F.col("location_name").rlike("^((?![Ss]anctuary).)*$")) &
    (F.col("location_name").rlike("^((?![Ee]quipping).)*$")) &
    (F.col("location_name").rlike("^((?![Jj]ohn).)*$")) &
    (F.col("location_name").rlike("^((?![Aa]ndrew).)*$")) &
    (F.col("location_name").rlike("^((?![Ee]manuel).)*$")) &
    (F.col("location_name").rlike("^((?![Rr]edeemed).)*$")) &
    (F.col("location_name").rlike("^((?![Pp]erfecting).)*$")) &
    (F.col("location_name").rlike("^((?![Aa]ngel).)*$")) &
    (F.col("location_name").rlike("^((?![Aa]rchangel).)*$")) &
    (F.col("location_name").rlike("^((?![Mm]icheal).)*$")) &
    (F.col("location_name").rlike("^((?![Tt]hought).)*$")) &
    (F.col("location_name").rlike("^((?![Pp]ariosse).)*$")) &
    (F.col("location_name").rlike("^((?![Cc]osmas).)*$")) &
    (F.col("location_name").rlike("^((?![Dd]eliverance).)*$")) &
    (F.col("location_name").rlike("^((?![Ss]ociete).)*$")) &
    (F.col("location_name").rlike("^((?![Tt]emple).)*$")) &
    (F.col("location_name").rlike("^((?![Ss]eminary).)*$")) &
    (F.col("location_name").rlike("^((?![Ee]mployment).)*$")) &
    (F.col("location_name").rlike("^((?![Ii]nstitute).)*$")) &
    (F.col("location_name").rlike("^((?![Cc]amp).)*$")) &
    (F.col("location_name").rlike("^((?![Ss]tudent).)*$")) &
    (F.col("location_name").rlike("^((?![Ee]ducation).)*$")) &
    (F.col("location_name").rlike("^((?![Ss]ocial).)*$")) &
    (F.col("location_name").rlike("^((?![Ww]welfare).)*$")) &
    (F.col("location_name").rlike("^((?![Cc][Ee][Ss]).)*$")) &
    (F.col("location_name").rlike("^((?![Ff]amily).)*$")) &
    (F.col("location_name").rlike("^((?![Mm]ary).)*$")) &
    (F.col("location_name").rlike("^((?![Rr]ussian).)*$")) &
    (F.col("location_name").rlike("^((?![Bb]eautif).)*$")) &
    (F.col("location_name").rlike("^((?![Hh]eaven).)*$")) &    
    (F.col("location_name").rlike("^((?!Inc).)*$")) &
    (F.col("location_name").rlike("^((?!God).)*$"))
  )
  
hathaway_churches.display()

placekey,poi_cbg,parent_placekey,location_name,brands,safegraph_brand_ids,store_id,top_category,sub_category,naics_code,open_hours,category_tags,latitude,longitude,street_address,city,region,postal_code,iso_country_code,opened_on,closed_on,tracking_closed_since,websites,phone_number,wkt_area_sq_meters
zzy-222@5wj-krc-dsq,160499400001,null,The Church of Jesus Christ of Latter day Saints,null,null,null,Religious Organizations,Religious Organizations,813110,null,Churches,45.927403,-116.11182,Valley Vw,Kamiah,ID,83536,US,null,null,2019-07,[],+12089352842,1362.0
zzw-222@5wq-s83-g8v,160679702001,null,The Church of Jesus Christ of Latter day Saints,null,null,null,Religious Organizations,Religious Organizations,813110,null,Churches,42.557902,-113.792354,241 N Overland Ave,Burley,ID,83318,US,null,null,2019-07,"[""lds.org""]",+12086780434,954.0
222-222@5wf-zyt-rhq,160439703001,null,The Church of Jesus Christ of Latter day Saints,null,null,null,Religious Organizations,Religious Organizations,813110,null,Churches,43.967664,-111.680578,145 E 1st N,St. Anthony,ID,83445,US,null,null,2019-07,"[""lds.org""]",+12086247540,1281.0
223-223@5w9-hwd-5mk,160010102252,null,The Church of Jesus Christ of Latter day Saints,null,null,null,Religious Organizations,Religious Organizations,813110,null,Churches,43.695074,-116.345195,700 E State St,Eagle,ID,83616,US,null,null,2019-07,[],+12089392988,522.0
zzy-222@5ws-j8j-6x5,160419701002,null,The Church of Jesus Christ of Latter day Saints,null,null,null,Religious Organizations,Religious Organizations,813110,null,Churches,42.100707,-111.871853,213 N 2nd E,Preston,ID,83263,US,null,null,2019-07,"[""lds.org""]",+12088521469,60.0
zzy-222@5wq-s89-9cq,null,null,The Church of Jesus Christ of Latter day Saints,null,null,null,Religious Organizations,Religious Organizations,813110,null,Churches,42.537956,-113.794962,213 W Main St,Burley,ID,83318,US,null,null,2019-07,"[""lds.org""]",+12086542591,923.0
222-222@5ws-hhd-zpv,null,null,The Church of Jesus Christ of Latter day Saints,null,null,null,Religious Organizations,Religious Organizations,813110,null,Churches,42.230853,-111.95055,7389 N 3000 W,Preston,ID,83263,US,null,null,2019-07,"[""lds.org""]",+12088522768,218.0
zzw-222@5wq-s83-g8v,null,null,The Church of Jesus Christ of Latter day Saints,null,null,null,Religious Organizations,Religious Organizations,813110,null,Churches,42.557902,-113.792354,241 N Overland Ave,Burley,ID,83318,US,null,null,2019-07,"[""lds.org""]",+12086780434,954.0
zzy-222@5wr-7dm-9xq,160830009001,null,The Church of Jesus Christ of Latter day Saints,null,null,null,Religious Organizations,Religious Organizations,813110,null,Churches,42.586804,-114.443502,2085 S Temple Dr,Twin Falls,ID,83301,US,null,null,2019-07,"[""mormon.org""]",+12087333446,2467.0
zzy-222@5w9-jbw-p7q,160010001001,null,The Church of Jesus Christ of Latter day Saints,null,null,null,Religious Organizations,Religious Organizations,813110,null,Churches,43.619514,-116.20184,Shamrock & Mcmillanrd,Boise,ID,83702,US,null,null,2019-07,"[""lds.org""]",+12083773220,1210.0


In [0]:
# Exclude the temples
churches = hathaway_churches.join(idaho_temples, hathaway_churches.placekey == idaho_temples.placekey, how='left_anti')

### 3. Diagram of tables and columns used to build the feature

In [0]:
# Diagram of Tables and Columns
print("Church Schema: ")
churches.select(
    'placekey',
    'location_name',
).printSchema()
print()
print("Patterns Schema: ")
patterns.select(
    'placekey',
    'date_range_start',
    'raw_visit_counts',
    'popularity_by_day',
    'visitor_home_aggregation',
    'normalized_visits_by_state_scaling',
).printSchema()

Church Schema: 
root
 |-- placekey: string (nullable = true)
 |-- location_name: string (nullable = true)


Patterns Schema: 
root
 |-- placekey: string (nullable = true)
 |-- date_range_start: string (nullable = true)
 |-- raw_visit_counts: double (nullable = true)
 |-- popularity_by_day: map (nullable = true)
 |    |-- key: string
 |    |-- value: integer (valueContainsNull = true)
 |-- visitor_home_aggregation: map (nullable = true)
 |    |-- key: string
 |    |-- value: integer (valueContainsNull = true)
 |-- normalized_visits_by_state_scaling: double (nullable = true)



### 4. Code Snippet of Data Wrangling

In [0]:
""" 
Assumptions: 
The state scaling ratio is accurate. 
"""
# Filter to just churches
church_patterns = patterns.join(churches, on='placekey', how='leftsemi')

# visitor_home_aggregation
home_agg = church_patterns.select(
    "*", 
    # Explodes the map of census tracts and visitor counts
    F.explode(
        F.col('visitor_home_aggregation')
        ).alias('tract', 'tract_visitors'),    
    # Gets the state scaling ratio
    (F.col('normalized_visits_by_state_scaling')/F.col('raw_visit_counts')).alias('state_scaling'),
    # Multiplies tract visitors by the state scaling ratio to get a more accurate total estimate
    (F.col('tract_visitors') * F.col('state_scaling')).alias('tract_visitors_scaled')
)

In [0]:
# Calculates the Sunday visits by tract
final_df = home_agg.filter(
    F.col('tract').startswith('16') # 16 is Idaho's tract number
    
).select(
    "*",
    F.month(home_agg['date_range_start']).alias('month')

).groupBy(
    'tract', 
    'month',

).agg(
    F.sum('tract_visitors_scaled').alias('sum_tract_visitors'),
    F.median('distance_from_home').alias('median_distance_from_home')

).groupBy(
    'tract'

).agg(
    F.min('sum_tract_visitors').alias('min_visitors'),
    F.max('sum_tract_visitors').alias('max_visitors'),
    F.median('sum_tract_visitors').alias('med_visitors'),
    F.mean('sum_tract_visitors').alias('mean_visitors'),
    F.stddev('sum_tract_visitors').alias('sd_visitors'),
    F.count('tract').alias('months'),
    F.lit('silicon_slayers').alias('team_name')


)

final_df.display()

tract,min_visitors,max_visitors,med_visitors,mean_visitors,sd_visitors,months,team_name
16001000100,65.62128572839684,356.03952935062404,192.6767738023044,190.99917904261346,108.04601572881236,11,silicon_slayers
16001000201,111.85507740750458,692.7386795530902,430.06426553672316,398.8264190306687,201.16493489182764,11,silicon_slayers
16001000202,131.24257145679368,1626.363917326637,773.8599107074779,885.5992910264547,458.1387182666222,11,silicon_slayers
16001000302,63.91718709000262,386.92995535373905,237.35968623374936,217.19071266116546,114.40614663803447,11,silicon_slayers
16001000303,61.43775221953189,306.86111647672396,199.5183725931651,184.2127072585722,90.303255197956,5,silicon_slayers
16001000304,131.2425714567937,1288.8161480057493,680.9967214225805,668.5779796662492,391.6516666007878,11,silicon_slayers
16001000400,147.6478928888929,539.5154807423296,257.7632296011499,287.52202621541824,124.5824459603369,11,silicon_slayers
16001000500,196.8638571851905,997.2986285493528,537.1648492892907,560.9212767291174,249.6800046071179,11,silicon_slayers
16001000600,65.62128572839684,628.2978721528027,368.6265133171913,370.15191177262926,190.86449069945417,11,silicon_slayers
16001000701,123.81758571319648,596.0774684526591,271.64804513251113,294.34961252496674,141.13442762265402,11,silicon_slayers


### 5. Visualizations

In [0]:
!pip install geopandas
!pip install folium

  Using cached geopandas-0.14.0-py3-none-any.whl (1.1 MB)
  Using cached fiona-1.9.5-cp39-cp39-manylinux2014_x86_64.whl (15.7 MB)
  Using cached shapely-2.0.2-cp39-cp39-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (2.5 MB)
  Using cached pyproj-3.6.1-cp39-cp39-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (8.3 MB)
  Using cached cligj-0.7.2-py3-none-any.whl (7.1 kB)
  Using cached click_plugins-1.1.1-py2.py3-none-any.whl (7.5 kB)
  Using cached importlib_metadata-6.8.0-py3-none-any.whl (22 kB)
  Using cached zipp-3.17.0-py3-none-any.whl (7.4 kB)
You should consider upgrading via the '/local_disk0/.ephemeral_nfs/envs/pythonEnv-97cb8c7c-a6b6-489f-ac99-c8f0961df55c/bin/python -m pip install --upgrade pip' command.
  Using cached folium-0.14.0-py2.py3-none-any.whl (102 kB)
  Using cached branca-0.6.0-py3-none-any.whl (24 kB)
You should consider upgrading via the '/local_disk0/.ephemeral_nfs/envs/pythonEnv-97cb8c7c-a6b6-489f-ac99-c8f0961df55c/bin/python -m pip install --upgrade pip' comm

In [0]:
import geopandas as gpd
import folium

dbutils.fs.cp("dbfs:/FileStore/tl_2019_16_tract", "file:/tmp/tl_2019_16_tract", recurse = True)
gdf = gpd.read_file("/tmp/tl_2019_16_tract")
# Join graph_df and gdf
gdf = gdf.merge(final_df.toPandas(), left_on='GEOID', right_on='tract')

In [0]:
# gdf
# Assuming gdf is your GeoPandas DataFrame containing census tracts
m = folium.Map(location=[45.697873, -114.346173], zoom_start=6)
# Convert the GeoDataFrame to a GeoJSON and add it to the folium map
gdf_json = gdf.to_json()
choropleth = folium.Choropleth(
    geo_data=gdf_json,
    data=gdf,
    columns=['tract', 'med_visitors'],  
    key_on='feature.properties.tract', 
    fill_color='YlGn', 
    fill_opacity=0.8,
    line_opacity=0.5,
    legend_name='Sunday Visits'
)

# tooltip = folium.GeoJsonTooltip(fields=['tract', 'sum_sunday_visitors'], aliases=['Tract', 'Visitors'], localize=True)

# choropleth.add_child(tooltip)
choropleth.add_to(m)

# m


Out[42]: <folium.features.Choropleth at 0x7f9e72668040>

<img src = '/files/Screenshot_2023_10_31_at_8_33_14_PM.png'>
<img src ='/files/newplot__1_.png'>


### 6. Display of the first five rows of your feature table used in the visualizations

In [0]:
display(final_df.orderBy('tract').head(5))

tract,min_visitors,max_visitors,med_visitors,mean_visitors,sd_visitors,months,team_name
16001000100,65.62128572839684,356.03952935062404,192.6767738023044,190.99917904261346,108.04601572881236,11,silicon_slayers
16001000201,111.85507740750458,692.7386795530902,430.06426553672316,398.8264190306687,201.16493489182764,11,silicon_slayers
16001000202,131.24257145679368,1626.363917326637,773.8599107074779,885.5992910264547,458.1387182666222,11,silicon_slayers
16001000302,63.91718709000262,386.92995535373905,237.35968623374936,217.19071266116546,114.40614663803447,11,silicon_slayers
16001000303,61.43775221953189,306.86111647672396,199.5183725931651,184.2127072585722,90.303255197956,5,silicon_slayers


In [0]:
final_df.agg(F.sum('max_visitors')).show()

+------------------+
| sum(max_visitors)|
+------------------+
|295317.13628566655|
+------------------+

